In [26]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from nemo import evaluate_nemo
import util
from topology import get_lognorm_hist
import topology
from util import evaluate
import pandas as pd
import random
from nemo import NemoSolver

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Topology Setup

In [50]:
def clean_df(df):
    min_s = df.isna().sum().min()
    df = df[df.isnull().sum(axis=1) < min_s]
    df = df[list(df.index)]
    df = df[df.isnull().sum(axis=1) < 10]
    df = df[list(df.index)]
    # set diag to 0
    for i in list(df.index):
        df.at[i, i] = 0

    # Make the DataFrame symmetric
    df = df + df.T - np.diag(df.values.diagonal())
    df = df.fillna(1)
    df = df.astype(int)
    return df


def prepare_all_dfs(root_dir, times=range(0, 23), days=range(1, 30)):
    outputs = []
    common_index = None
    for time in times:
        for day in days:
            loc = root_dir + f"time-{time}-day-{day}.csv"
            print(loc)
            df_plb = pd.read_csv(loc, sep=",", header=None, dtype=float)
            df_plb = df_plb.astype(float)
            df_plb = df_plb.iloc[:, :-1]
            df_plb.replace(-1, pd.NA, inplace=True)
            df_plb = clean_df(df_plb)

            if common_index is None:
                common_index = set(df_plb.index)
            else:
                common_index = common_index & set(df_plb.index)
            outputs.append(df_plb)

    for i, df in enumerate(outputs):
        df = df[df.index.isin(list(common_index))]
        df = df[list(df.index)]
        outputs[i] = df
    return outputs

In [52]:
offset = 3
df_months = prepare_all_dfs("datasets/RIPEAtlas/month/", times=range(0, 24), days=range(1, 2))

datasets/RIPEAtlas/month/time-0-day-1.csv
datasets/RIPEAtlas/month/time-1-day-1.csv
datasets/RIPEAtlas/month/time-2-day-1.csv
datasets/RIPEAtlas/month/time-3-day-1.csv
datasets/RIPEAtlas/month/time-4-day-1.csv
datasets/RIPEAtlas/month/time-5-day-1.csv
datasets/RIPEAtlas/month/time-6-day-1.csv
datasets/RIPEAtlas/month/time-7-day-1.csv
datasets/RIPEAtlas/month/time-8-day-1.csv
datasets/RIPEAtlas/month/time-9-day-1.csv
datasets/RIPEAtlas/month/time-10-day-1.csv
datasets/RIPEAtlas/month/time-11-day-1.csv
datasets/RIPEAtlas/month/time-12-day-1.csv
datasets/RIPEAtlas/month/time-13-day-1.csv
datasets/RIPEAtlas/month/time-14-day-1.csv
datasets/RIPEAtlas/month/time-15-day-1.csv
datasets/RIPEAtlas/month/time-16-day-1.csv
datasets/RIPEAtlas/month/time-17-day-1.csv
datasets/RIPEAtlas/month/time-18-day-1.csv
datasets/RIPEAtlas/month/time-19-day-1.csv
datasets/RIPEAtlas/month/time-20-day-1.csv
datasets/RIPEAtlas/month/time-21-day-1.csv
datasets/RIPEAtlas/month/time-22-day-1.csv
datasets/RIPEAtlas/mo

In [59]:
for i, df in enumerate(df_months):
    loc = f"datasets/RIPEAtlas/02072023/time-{i}.csv"
    df.to_csv(loc, header=True, index=False)

In [58]:
df_months[5]

,0,2,4,8,10,14,16,18,28,32,...,1797,1799,1803,1805,1807,1809,1811,1813,1815,1819
0,0,20,59,38,41,61,382,391,175,31,...,55,364,32,195,31,33,289,172,321,44
2,20,0,42,40,37,43,341,371,182,19,...,4,256,33,155,36,21,274,172,289,4
4,59,42,0,9,48,59,516,414,259,42,...,53,328,24,224,10,43,308,218,375,49
8,38,40,9,0,41,58,503,408,187,20,...,37,288,10,198,1,30,302,193,294,36
10,41,37,48,41,0,64,402,395,333,25,...,41,275,55,225,43,34,286,268,335,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,33,21,43,30,34,55,426,384,195,23,...,21,289,20,197,36,0,306,185,339,21
1811,289,274,308,302,286,390,403,635,143,314,...,274,137,324,143,371,306,0,126,50,275
1813,172,172,218,193,268,199,290,513,42,184,...,177,131,186,25,198,185,126,0,167,177
1815,321,289,375,294,335,349,429,654,132,328,...,316,8,364,155,356,339,50,167,0,302


In [2]:
df_plb = pd.read_csv("datasets/RIPEAtlas/month/time-0-day-2.csv", sep=",", header=None)
df_plb = df_plb.astype(float)
df_plb = df_plb.iloc[:, :-1]
df_plb.replace(-1, pd.NA, inplace=True)

In [3]:
df_plb

,0,1,2,3,4,5,6,7,8,9,...,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820
0,2.913954,<NA>,10.071039,<NA>,29.505642,<NA>,<NA>,<NA>,18.801541,<NA>,...,145.187708,<NA>,86.253351,<NA>,160.476427,<NA>,283.604187,<NA>,22.406282,<NA>
1,<NA>,3.124919,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,31.71046,...,<NA>,143.615587,<NA>,86.861502,<NA>,141.726601,<NA>,311.6265,<NA>,11.912735
2,9.972284,<NA>,1.029232,<NA>,21.127371,<NA>,<NA>,<NA>,16.973022,<NA>,...,137.07206,<NA>,86.498679,<NA>,144.907396,<NA>,289.20014,<NA>,2.252551,<NA>
3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,29.247761,<NA>,20.923276,<NA>,11.09714,<NA>,<NA>,<NA>,4.941659,<NA>,...,181.758786,<NA>,109.7656,<NA>,187.73926,<NA>,306.365028,<NA>,24.805953,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,<NA>,141.861072,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,148.391652,...,<NA>,15.103461,<NA>,82.914086,<NA>,0.082916,<NA>,452.335776,<NA>,168.034613
1817,281.597683,<NA>,284.566204,<NA>,319.512377,<NA>,<NA>,<NA>,<NA>,<NA>,...,453.691162,<NA>,374.195363,<NA>,476.007024,<NA>,0.391832,<NA>,310.716693,<NA>
1818,<NA>,296.533009,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,315.658193,...,<NA>,430.793238,<NA>,395.721508,<NA>,446.821279,<NA>,0.426806,<NA>,283.106903
1819,22.244281,<NA>,1.860806,<NA>,24.698623,<NA>,<NA>,<NA>,18.271304,<NA>,...,137.812962,<NA>,86.559507,<NA>,151.267923,<NA>,286.589017,<NA>,0.358726,<NA>


In [89]:
df_sums = df_plb.isna().sum()
print("Min:", df_sums.min(), "Max:", df_sums.max())

Min: 1085 Max: 1821


In [90]:
df1 = df_plb[df_plb.isnull().sum(axis=1) < 1084]
df1 = df1[list(df1.index)]
df1

,0,2,4,8,10,14,16,18,28,32,...,1801,1803,1805,1807,1809,1811,1813,1815,1817,1819
0,2.913954,10.071039,29.505642,18.801541,19.191051,31.39236,191.378631,189.950013,87.314931,15.539913,...,195.977178,16.727957,87.542701,16.142928,17.214772,145.187708,86.253351,160.476427,283.604187,22.406282
2,9.972284,1.029232,21.127371,16.973022,17.491243,22.282937,166.489693,185.599561,90.973138,9.882766,...,185.463103,15.92909,75.924789,18.48951,10.764563,137.07206,86.498679,144.907396,289.20014,2.252551
4,29.247761,20.923276,11.09714,4.941659,32.642098,29.592434,254.398678,201.978211,129.799935,21.301142,...,203.998412,12.196507,111.370319,5.079655,21.53743,181.758786,109.7656,187.73926,306.365028,24.805953
8,18.936559,16.774484,4.392463,0.37252,19.467639,29.322996,249.306347,204.216303,93.678034,9.240315,...,197.73808,5.315531,98.644062,0.791822,15.355092,151.36443,96.687373,147.407936,<NA>,18.230134
10,19.429173,16.665339,31.476769,19.035095,1.558739,35.774497,199.050446,191.844434,166.372578,30.200747,...,197.09052,27.753148,112.024551,21.858635,21.103499,143.259807,134.539447,167.516794,310.941731,18.675375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1811,145.221231,137.132743,154.458585,151.450426,147.750125,152.277354,198.03038,312.007895,71.735561,157.182623,...,275.991919,162.418104,70.869337,150.919812,153.254686,0.15627,63.431075,17.446089,433.119594,137.949303
1813,86.363699,86.334736,109.128806,96.722848,134.581554,99.75352,141.859283,250.889705,21.027011,92.292562,...,205.191284,93.000125,11.861157,99.471062,92.990722,63.332708,0.166431,83.828111,363.218699,88.472277
1815,160.736795,144.757579,187.745818,147.304155,167.621886,170.521891,214.002262,328.895353,72.028951,163.713078,...,304.943209,182.0955,78.064285,180.690315,170.003573,313.834642,83.88138,0.073838,466.317125,151.185795
1817,281.597683,284.566204,319.512377,<NA>,315.030215,312.248671,473.804916,478.642924,367.064861,320.329602,...,495.939886,284.886953,394.311516,307.743943,282.363412,453.691162,374.195363,476.007024,0.391832,310.716693


In [91]:
df1.isna().sum().max()

466

In [92]:
df2 = df1[df1.isnull().sum(axis=1) < 10]
df2 = df2[list(df2.index)]
df2

,0,4,8,10,14,16,18,28,32,34,...,1799,1801,1803,1805,1807,1809,1811,1813,1815,1819
0,2.913954,29.505642,18.801541,19.191051,31.39236,191.378631,189.950013,87.314931,15.539913,10.656341,...,136.391938,195.977178,16.727957,87.542701,16.142928,17.214772,145.187708,86.253351,160.476427,22.406282
4,29.247761,11.09714,4.941659,32.642098,29.592434,254.398678,201.978211,129.799935,21.301142,20.49693,...,164.8473,203.998412,12.196507,111.370319,5.079655,21.53743,181.758786,109.7656,187.73926,24.805953
8,18.936559,4.392463,0.37252,19.467639,29.322996,249.306347,204.216303,93.678034,9.240315,21.714937,...,144.292132,197.73808,5.315531,98.644062,0.791822,15.355092,151.36443,96.687373,147.407936,18.230134
10,19.429173,31.476769,19.035095,1.558739,35.774497,199.050446,191.844434,166.372578,30.200747,18.570236,...,138.299864,197.09052,27.753148,112.024551,21.858635,21.103499,143.259807,134.539447,167.516794,18.675375
14,30.56668,28.814195,28.902356,34.563788,1.492159,248.976469,206.054343,96.671647,21.741862,24.170252,...,150.506435,275.626986,28.104685,93.566142,38.413028,27.9931,152.035014,99.689137,170.35415,22.839008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,17.004509,21.460963,15.359472,21.199179,28.35432,210.358917,186.181049,97.487742,11.706518,12.375446,...,144.896892,185.88898,10.422117,97.884005,18.541682,0.323176,153.2332,92.986052,170.016688,10.781093
1811,145.221231,154.458585,151.450426,147.750125,152.277354,198.03038,312.007895,71.735561,157.182623,148.054415,...,20.775288,275.991919,162.418104,70.869337,150.919812,153.254686,0.15627,63.431075,17.446089,137.949303
1813,86.363699,109.128806,96.722848,134.581554,99.75352,141.859283,250.889705,21.027011,92.292562,83.501548,...,65.574167,205.191284,93.000125,11.861157,99.471062,92.990722,63.332708,0.166431,83.828111,88.472277
1815,160.736795,187.745818,147.304155,167.621886,170.521891,214.002262,328.895353,72.028951,163.713078,162.562826,...,3.974632,304.943209,182.0955,78.064285,180.690315,170.003573,313.834642,83.88138,0.073838,151.185795


In [93]:
for i in list(df2.index):
    df2.at[i, i] = 0

# Make the DataFrame symmetric
df2 = df2 + df2.T - np.diag(df2.values.diagonal())
df2

,0,4,8,10,14,16,18,28,32,34,...,1799,1801,1803,1805,1807,1809,1811,1813,1815,1819
0,0,58.753404,37.7381,38.620224,61.95904,382.748922,379.978861,174.857152,31.065741,21.477687,...,271.331891,391.057784,33.199512,174.9268,32.229541,34.21928,290.40894,172.61705,321.213222,44.650563
4,58.753404,0,9.334122,64.118867,58.406629,510.287636,402.455577,259.320557,42.525406,40.899082,...,328.861231,407.73298,24.200211,222.572457,42.50169,42.998393,336.217371,218.894406,375.485078,49.504576
8,37.7381,9.334122,0,38.502734,58.225352,506.221148,407.954283,187.378382,18.451601,41.337032,...,288.560848,395.980966,10.587644,197.327322,1.729967,30.714564,302.814857,193.410221,294.712092,36.501438
10,38.620224,64.118867,38.502734,0,70.338285,406.084158,387.505961,333.242231,60.815649,37.379518,...,276.436421,394.611483,55.459561,224.160272,43.77161,42.302678,291.009932,269.121001,335.138681,36.562542
14,61.95904,58.406629,58.225352,70.338285,0,497.882593,418.388432,208.521983,43.758612,48.610748,...,301.026095,557.478839,56.288639,187.290054,76.982843,56.34742,304.312368,199.442657,340.876041,45.788177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,34.21928,42.998393,30.714564,42.302678,56.34742,420.880811,371.940952,195.013845,23.449868,24.743231,...,289.875094,371.844858,20.772921,195.794318,37.104809,0,306.487886,185.976774,340.020261,21.611645
1811,290.40894,336.217371,302.814857,291.009932,304.312368,403.434837,623.639671,143.457319,314.354499,296.174431,...,41.456818,551.885274,324.809914,141.680924,301.746185,306.487886,0,126.763783,331.280731,275.762265
1813,172.61705,218.894406,193.410221,269.121001,199.442657,284.514238,502.378512,42.115798,184.677684,196.08463,...,131.222261,410.39043,185.902623,23.780437,198.965092,185.976774,126.763783,0,167.709491,175.031785
1815,321.213222,375.485078,294.712092,335.138681,340.876041,432.567119,657.605991,144.214503,327.42466,325.109057,...,7.991953,609.917977,364.152132,156.133016,361.398797,340.020261,331.280731,167.709491,0,302.453718


In [95]:
df3 = df2.fillna(1)
df3 = df3.astype(int)
df3

,0,4,8,10,14,16,18,28,32,34,...,1799,1801,1803,1805,1807,1809,1811,1813,1815,1819
0,0,58,37,38,61,382,379,174,31,21,...,271,391,33,174,32,34,290,172,321,44
4,58,0,9,64,58,510,402,259,42,40,...,328,407,24,222,42,42,336,218,375,49
8,37,9,0,38,58,506,407,187,18,41,...,288,395,10,197,1,30,302,193,294,36
10,38,64,38,0,70,406,387,333,60,37,...,276,394,55,224,43,42,291,269,335,36
14,61,58,58,70,0,497,418,208,43,48,...,301,557,56,187,76,56,304,199,340,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,34,42,30,42,56,420,371,195,23,24,...,289,371,20,195,37,0,306,185,340,21
1811,290,336,302,291,304,403,623,143,314,296,...,41,551,324,141,301,306,0,126,331,275
1813,172,218,193,269,199,284,502,42,184,196,...,131,410,185,23,198,185,126,0,167,175
1815,321,375,294,335,340,432,657,144,327,325,...,7,609,364,156,361,340,331,167,0,302
